In [9]:
import pandas as pd
import numpy as np
import re
import torch
from torch import nn
import nltk
import pandas as pd
import matplotlib.pyplot as plt
from sentence_transformers import *
import torchvision.models as models


import torch.nn as nn
from transformers import BertModel
from transformers import BertTokenizer
import torch.nn as nn
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer, util

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/vscode/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
training_images_path = '/workspaces/Shopee-Price-Match-Guarantee/00_source_data/shopee-product-matching/train_images'
training_dataset =pd.read_csv('/workspaces/Shopee-Price-Match-Guarantee/00_source_data/shopee-product-matching/train.csv')
testing_dataset = pd.read_csv('/workspaces/Shopee-Price-Match-Guarantee/00_source_data/shopee-product-matching/test.csv')

In [3]:
# remove stop words 
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# remove punctuation
import string
punctuations = string.punctuation

# remove numbers
import re
def remove_numbers(text):
    result = re.sub(r'\d+', '', text)
    return result

# remove special characters
def remove_special_characters(text):
    pattern = r'[^a-zA-z0-9\s]'
    text = re.sub(pattern, '', text)
    return text

# remove extra spaces
def remove_extra_spaces(text):
    text = re.sub(' +', ' ', text)
    return text

def word_tokenize(text):
    text = text.split()
    return text

# remove stop words
def remove_stop_words(text):
    text_tokens = word_tokenize(text)
    tokens_without_sw = [word for word in text_tokens if not word in stop_words]
    filtered_sentence = (" ").join(tokens_without_sw)
    return filtered_sentence

# remove all preprocessing
def remove_all_preprocessing(text):
    text = remove_numbers(text)
    text = remove_special_characters(text)
    text = remove_extra_spaces(text)
    text = remove_stop_words(text)
    return text

# apply all preprocessing
training_dataset['title'] = training_dataset['title'].apply(lambda x: remove_all_preprocessing(x))
testing_dataset['title'] = testing_dataset['title'].apply(lambda x: remove_all_preprocessing(x))

In [4]:
# get rid of \
training_dataset['title'] = training_dataset['title'].apply(lambda x: x.replace('\\', ''))
# lower case
training_dataset['title'] = training_dataset['title'].apply(lambda x: x.lower())

In [5]:
model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')
embeddings = model.encode(training_dataset['title'])
print(embeddings)

[[-0.05237605  0.00266704 -0.0351623  ... -0.0220446   0.03120822
   0.0482746 ]
 [ 0.03168393  0.0207246  -0.01772413 ... -0.02363313  0.00672482
  -0.06600535]
 [-0.02553968  0.02782196  0.01990272 ... -0.0465625  -0.01983832
   0.01210797]
 ...
 [-0.00043891  0.0663081  -0.00434749 ...  0.04577914  0.02965026
  -0.03386611]
 [-0.02243138  0.02246639  0.05454137 ...  0.01867248  0.01682098
   0.00408043]
 [-0.03517014  0.02272914  0.00684843 ... -0.00679976 -0.01856173
  -0.00943737]]


In [6]:
from sentence_transformers import SentenceTransformer, models
word_embedding_model = models.Transformer('sentence-transformers/distiluse-base-multilingual-cased-v2', max_seq_length=256)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), out_features=256, activation_function=nn.Tanh())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])


In [12]:
bert_version = 'sentence-transformers/distiluse-base-multilingual-cased-v2'
tokenizer = BertTokenizer.from_pretrained(bert_version)
model = BertModel.from_pretrained(bert_version)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at sentence-transformers/distiluse-base-multilingual-cased-v2 were not used when initializing BertModel: ['transformer.layer.2.attention.k_lin.weight', 'transformer.layer.2.ffn.lin2.bias', 'transformer.layer.0.ffn.lin2.bias', 'transformer.layer.5.sa_layer_norm.weight', 'transformer.layer.1.output_layer_norm.weight', 'transformer.layer.2.attention.out_lin.bias', 'transformer.layer.3.sa_layer_norm.weight', 'transformer.layer.3.attention.q_lin.bias', 'transformer.layer.5.attention.v_lin.weight', 'transformer.laye

In [7]:
from tqdm.auto import tqdm
from torchmetrics.text.bert import BERTScore

progress_bar_train = tqdm(range(num_training_steps))
progress_bar_eval = tqdm(range(25 * len(eval_dataloader)))


for epoch in range(25):
  model.train()
  for batch in train_dataloader:
      batch = {k: v.to(device) for k, v in batch.items()}
      outputs = model(**batch)
      loss = outputs.loss
      loss.backward()

      optimizer.step()
      lr_scheduler.step()
      optimizer.zero_grad()
      progress_bar_train.update(1)

  model.eval()
  for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])
    progress_bar_eval.update(1)
    
  print(metric.compute())

NameError: name 'num_training_steps' is not defined